In [ ]:
import pandas as pd
import json
import glob
import tqdm
import matplotlib.pyplot as plt
import numpy as np

import sklearn
import sklearn.metrics
import matplotlib
import scipy
import mplhep
import os
import awkward

import vector
import fastjet
import awkward as ak

import pandas
import boost_histogram as bh
import itertools
import mplhep
import numba

mplhep.set_style(mplhep.styles.CMS)

In [ ]:
import sys

sys.path += ["../../mlpf/plotting/"]

import plot_utils
from plot_utils import pid_to_text, load_eval_data, compute_jet_ratio

In [ ]:
yvals, X, _ = load_eval_data(
    "../../experiments/mlpf-clic-2023-results/clusters_best_tuned_gnn_clic_v130/"+
    "evaluation/epoch_96/clic_edm_ttbar_pu10_pf/test/*.parquet", -1)

In [ ]:
b = np.linspace(0,700,50)
plt.hist(ak.sum(X[:, :, 0]!=0, axis=1), bins=b)
plt.axvline(256, color="black")
plt.yscale("log")
plt.xlabel("Number of elements per event")

In [ ]:
msk_evs_big = ak.sum(X[:, :, 0]!=0, axis=1)>256
msk_has_mu = ak.sum(yvals["gen_cls_id"]==5, axis=1)>=1

In [ ]:
jet_vec = vector.awk(
    awkward.zip(
        {
            "pt": yvals["jets_gen_pt"],
            "eta": yvals["jets_gen_eta"],
            "phi": yvals["jets_gen_phi"],
            "energy": yvals["jets_gen_energy"],
        }
    )
)

msk_lep = (yvals["gen_cls_id"]==4) | (yvals["gen_cls_id"]==5)
lep_vec = vector.awk(
    awkward.zip(
        {
            "pt": yvals["gen_pt"][msk_lep],
            "eta": yvals["gen_eta"][msk_lep],
            "phi": yvals["gen_phi"][msk_lep],
            "energy": yvals["gen_energy"][msk_lep],
        }
    )
)

In [ ]:
jet_combs = awkward.combinations(jet_vec, 2)
jet_lep_combs = awkward.cartesian([jet_vec, lep_vec])
jet_lep_combs_arg = awkward.argcartesian([jet_vec, lep_vec])

In [ ]:
def deltaphi(phi1, phi2):
    diff = phi1 - phi2
    return np.arctan2(np.sin(diff), np.cos(diff))

def deltar(eta1, phi1, eta2, phi2):
    deta = eta1 - eta2
    dphi = deltaphi(phi1, phi2)
    return np.sqrt(deta**2 + dphi**2)

In [ ]:
drs = deltar(
    jet_combs['0'].eta,
    jet_combs['0'].phi,
    jet_combs['1'].eta,
    jet_combs['1'].phi
)

drs_jet_lep = deltar(
    jet_lep_combs['0'].eta,
    jet_lep_combs['0'].phi,
    jet_lep_combs['1'].eta,
    jet_lep_combs['1'].phi
)

In [ ]:
gen_pt_vals = yvals["gen_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep<0.5]]
cand_pt_vals = yvals["cand_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep<0.5]]
pred_pt_vals = yvals["pred_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep<0.5]]

b = np.linspace(0,200,100)
plt.hist(awkward.flatten(gen_pt_vals[gen_pt_vals>0]), bins=b, histtype="step", lw=2, label="gen");
plt.hist(awkward.flatten(cand_pt_vals[cand_pt_vals>0]), bins=b, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(pred_pt_vals[pred_pt_vals>0]), bins=b, histtype="step", lw=2, label="MLPF");
plt.legend(title="ch.leptons, $\Delta R_{jet} < 0.5$")
plt.yscale("log")
plt.xlabel("particle $p_T$ [GeV]")
plt.ylabel("Number of particles / bin")
plt.savefig("lepton_pt_dr_less0p5.pdf")

In [ ]:
gen_pt_vals = yvals["gen_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep>0.5]]
cand_pt_vals = yvals["cand_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep>0.5]]
pred_pt_vals = yvals["pred_pt"][msk_lep][jet_lep_combs_arg['1'][drs_jet_lep>0.5]]

b = np.linspace(0,200,100)
plt.hist(awkward.flatten(gen_pt_vals[gen_pt_vals>0]), bins=b, histtype="step", lw=2, label="gen");
plt.hist(awkward.flatten(cand_pt_vals[cand_pt_vals>0]), bins=b, histtype="step", lw=2, label="PF");
plt.hist(awkward.flatten(pred_pt_vals[pred_pt_vals>0]), bins=b, histtype="step", lw=2, label="MLPF");
plt.legend(title="ch.leptons, $\Delta R_{jet} > 0.5$")
plt.yscale("log")
plt.xlabel("particle $p_T$ [GeV]")
plt.ylabel("Number of particles / bin")
plt.savefig("lepton_pt_dr_greater0p5.pdf")

In [ ]:
b = np.linspace(0,5,100)
plt.hist(awkward.flatten(drs), bins=b, histtype="step", lw=2, label="gen.jet-gen.jet $\Delta R$");
plt.hist(awkward.flatten(drs_jet_lep), bins=b, histtype="step", lw=2, label="gen.jet-gen.lepton $\Delta R$");
plt.legend()
plt.xlabel("$\Delta R$")
plt.ylabel("pairs / bin")
plt.savefig("jet_lepton_dr.pdf")